In [67]:
import os

In [68]:
%pwd

'/Users/pradhyumnyadav'

In [69]:
os.chdir("../")

In [70]:
%pwd

'/Users'

In [71]:
# from dataclasses import dataclass
# from pathlib import Path
# import os
# import urllib.request as request
# from zipfile import ZipFile
# import tensorflow as tf
# #from ct_scan.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
# from ct_scan.utils.common import read_yaml, create_directories

from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf


In [72]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [73]:
class ConfigurationManager:
    def __init__(self):
        self.config = {
            "artifacts_root": "artifacts",
            "data_ingestion": {
                "root_dir": "artifacts/data_ingestion",
                "source_URL": "https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing",
                "local_data_file": "artifacts/data_ingestion/data.zip",
                "unzip_dir": "artifacts/data_ingestion"
            },
            "prepare_base_model": {
                "root_dir": "artifacts/prepare_base_model",
                "base_model_path": "artifacts/prepare_base_model/base_model.h5",
                "updated_base_model_path": "artifacts/prepare_base_model/base_model_updated.h5"
            },
            "training": {
                "root_dir": "artifacts/training",
                "trained_model_path": "artifacts/training/model.h5"
            }
        }
        self.params = {
            "AUGMENTATION": True,
            "IMAGE_SIZE": [224, 224, 3],
            "BATCH_SIZE": 16,
            "INCLUDE_TOP": False,
            "EPOCHS": 5,
            "CLASSES": 2,
            "WEIGHTS": "imagenet",
            "LEARNING_RATE": 0.01
        }

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config["prepare_base_model"]
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=config["root_dir"],
            base_model_path=config["base_model_path"],
            updated_base_model_path=config["updated_base_model_path"],
            params_image_size=self.params["IMAGE_SIZE"],
            params_learning_rate=self.params["LEARNING_RATE"],
            params_include_top=self.params["INCLUDE_TOP"],
            params_weights=self.params["WEIGHTS"],
            params_classes=self.params["CLASSES"]
        )

        return prepare_base_model_config



In [74]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


In [75]:
try:
    config_manager = ConfigurationManager()
    prepare_base_model_config = config_manager.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    print(f"An error occurred: {str(e)}")


[2024-02-23 12:44:40,296: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
An error occurred: artifacts; Permission denied
